In [2]:
import pandas as pd
import numpy as np
import torch
import random
import os
import sys

# ==========================================
# 1. 라이브러리 및 환경 설정
# ==========================================
try:
    from sklearn.metrics import classification_report, accuracy_score, f1_score
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
    from datasets import Dataset
    from sklearn.model_selection import train_test_split
except ImportError:
    sys.exit("❌ 라이브러리가 설치되지 않았습니다. !pip install transformers datasets accelerate scikit-learn 실행 필요")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 [KoELECTRA] 사용 장치: {device}")

# ==========================================
# 2. 데이터 로드 및 병합
# ==========================================
url = "https://raw.githubusercontent.com/tunib-ai/DKTC/main/data/train.csv"
try:
    df_threat = pd.read_csv(url)[['class', 'conversation']]
    print(f"✅ 위협 데이터 로드 완료: {len(df_threat)}개")
except Exception as e:
    sys.exit(f"❌ 위협 데이터 로드 실패: {e}")

# 일반 대화 파일 로드 (우선순위: 업로드된 파일명)
normal_file = "normal_conversation (1).csv"
if not os.path.exists(normal_file):
    # 대체 파일명 확인 (괄호 없는 버전)
    if os.path.exists("normal_conversation.csv"):
        normal_file = "normal_conversation.csv"
    else:
        sys.exit(f"❌ '{normal_file}' 파일을 찾을 수 없습니다.")

print(f"📂 사용할 일반 대화 파일: {normal_file}")
df_normal = pd.read_csv(normal_file)
df_normal['conversation'] = df_normal['conversation'].str.replace(r'(^|\n)[AB]:\s*', '', regex=True)

# 컬럼명 통일
if 'class' not in df_normal.columns:
    df_normal['class'] = '일반 대화'
df_normal = df_normal[['class', 'conversation']]
print(f"✅ 일반 대화 데이터 로드 완료: {len(df_normal)}개")

# 병합 및 셔플
df_final = pd.concat([df_threat, df_normal], ignore_index=True)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"📊 최종 학습 데이터 개수: {len(df_final)}개")

# ==========================================
# 3. 라벨 인코딩 & 데이터셋 변환
# ==========================================
label_map = {
    '협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3,
    '협박': 0, '갈취': 1, '직장 내 괴롭힘': 2, '기타 괴롭힘': 3,
    '일반 대화': 4
}
df_final['label'] = df_final['class'].map(label_map)
df_final = df_final.dropna(subset=['label'])
df_final['label'] = df_final['label'].astype(int)

train_df, val_df = train_test_split(df_final, test_size=0.2, random_state=42, stratify=df_final['label'])
train_dataset = Dataset.from_pandas(train_df[['conversation', 'label']])
val_dataset = Dataset.from_pandas(val_df[['conversation', 'label']])

# ==========================================
# 4. 모델 & 토크나이저 (KoELECTRA)
# ==========================================
# 🚨 [변경됨] KoELECTRA 모델 사용 (-discriminator 필수!)
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
print(f"🔄 모델 로딩 중: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5).to(device)

def preprocess_function(examples):
    return tokenizer(examples["conversation"], truncation=True, max_length=256)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==========================================
# 5. 학습 (Training)
# ==========================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results_ko",     # [변경됨] 결과 폴더명
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",  
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,   
    compute_metrics=compute_metrics
)

print("\n📘 KoELECTRA 학습 시작! (약 5~10분 소요)")
trainer.train()

# ==========================================
# 6. 모델 저장
# ==========================================
SAVE_PATH = "./final_model_ko" # [변경됨] 저장 폴더명
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"\n✅ 학습 완료! 모델이 '{SAVE_PATH}' 폴더에 저장되었습니다.")

🚀 [KoELECTRA] 사용 장치: cuda
✅ 위협 데이터 로드 완료: 3950개
📂 사용할 일반 대화 파일: normal_conversation.csv
✅ 일반 대화 데이터 로드 완료: 800개
📊 최종 학습 데이터 개수: 4750개
🔄 모델 로딩 중: monologg/koelectra-base-v3-discriminator


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

ElectraForSequenceClassification LOAD REPORT from: monologg/koelectra-base-v3-discriminator
Key                                               | Status     | 
--------------------------------------------------+------------+-
discriminator_predictions.dense_prediction.weight | UNEXPECTED | 
discriminator_predictions.dense.bias              | UNEXPECTED | 
electra.embeddings.position_ids                   | UNEXPECTED | 
discriminator_predictions.dense.weight            | UNEXPECTED | 
discriminator_predictions.dense_prediction.bias   | UNEXPECTED | 
classifier.dense.bias                             | MISSING    | 
classifier.out_proj.weight                        | MISSING    | 
classifier.dense.weight                           | MISSING    | 
classifier.out_proj.bias                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missin

Map:   0%|          | 0/3800 [00:00<?, ? examples/s]

Map:   0%|          | 0/950 [00:00<?, ? examples/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['electra.embeddings.LayerNorm.weight', 'electra.embeddings.LayerNorm.bias', 'electra.encoder.layer.0.attention.output.LayerNorm.weight', 'electra.encoder.layer.0.attention.output.LayerNorm.bias', 'electra.encoder.layer.0.output.LayerNorm.weight', 'electra.encoder.layer.0.output.LayerNorm.bias', 'electra.encoder.layer.1.attention.output.LayerNorm.weight', 'electra.encoder.layer.1.attention.output.LayerNorm.bias', 'electra.encoder.layer.1.output.LayerNorm.weight', 'electra.encoder.layer.1.output.LayerNorm.bias', 'electra.encoder.layer.2.attention.output.LayerNorm.weight', 'electra.encoder.layer.2.attention.output.LayerNorm.bias', 'electra.encoder.layer.2.output.LayerNorm.weight', 'electra.encoder.layer.2.output.LayerNorm.bias', 'electra.encoder.layer.3.attention.output.LayerNorm.weight', 'electra.encoder.layer.3.attention.output.LayerNorm.bias', 'electra.encoder.layer.3.output.LayerNorm.weight', 'electra.encoder.layer.3.output.Laye

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]


✅ 학습 완료! 모델이 './final_model_ko' 폴더에 저장되었습니다.


In [5]:
import pandas as pd
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm


# 1. 설정 및 데이터 로드
MODEL_PATH = "./final_model_ko"       # 학습된 모델 경로
TEST_FILE = "test.json"            # 업로드된 테스트 파일

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 장치: {device}")

# 테스트 데이터 로드 (JSON)
try:
    with open(TEST_FILE, 'r', encoding='utf-8') as f:
        test_data = json.load(f)
    print(f"테스트 데이터 로드 완료: {len(test_data)}개")
    
    # 데이터 구조 확인 (첫 번째 아이템)
    first_key = list(test_data.keys())[0]
    print(f"데이터 예시: ID={first_key}, 내용={test_data[first_key]}")

except FileNotFoundError:
    print("오류: 'test.json' 파일을 찾을 수 없습니다.")
    # (테스트용 더미 데이터)
    test_data = {"t_000": {"text": "이거 들어봐 와 이 노래 진짜 좋다"}, "t_001": {"text": "야 돈 내놔"}}


# 2. 모델 로드
print("모델 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
except OSError:
    print("오류: 저장된 모델을 찾을 수 없습니다. 학습 코드를 먼저 실행했는지 확인해주세요.")
    # (임시 모델 로드)
    tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
    model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=5).to(device)

# 3. 추론 (Inference)
print("예측 시작...")

results = []

# test_data가 딕셔너리 형태 {"t_000": {"text": "..."}} 라고 가정
for idx, item in tqdm(test_data.items()):
    # 텍스트 추출 (text 키가 없으면 conversation 등 다른 키 시도)
    text = item.get('text', item.get('conversation', ''))
    
    # 전처리 및 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        
    # 예측값 (0~4)
    pred_label = torch.argmax(logits, dim=-1).item()
    
    # 결과 저장
    results.append({
        'idx': idx,
        'class': pred_label  # 숫자로 저장 (0, 1, 2, 3, 4)
    })

# 4. 제출 파일 생성
submission = pd.DataFrame(results)

# 컬럼 순서 정렬 (idx, class)
submission = submission[['idx', 'class']]

# 파일 저장
save_path = "submission.csv"
submission.to_csv(save_path, index=False)

print("\n" + "="*50)
print(f"제출 파일 생성 완료: {save_path}")
print("="*50)
print(submission.head())

사용 장치: cuda
테스트 데이터 로드 완료: 500개
데이터 예시: ID=t_000, 내용={'text': '아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그럼 취소할까요 아가씨 내 여기단골이니 담에 갖다줄께 저도 알바생이라 외상안됩니다 아따 누가 떼먹는다고 그러나 갖다준다고 안됩니다 자꾸이럼 경찰불러요 아가씨 담배피는교 그건 왜 물으세요 그람 아가씨 담배 한대만 빌립시다 내 지금 지갑도 잃어버리고 기분이 그래서 그러니 여기요  아따 주는김에 한개더 주면 되겠네'}
모델 로딩 중...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

예측 시작...


100%|██████████| 500/500 [00:05<00:00, 91.38it/s]


제출 파일 생성 완료: submission.csv
     idx  class
0  t_000      1
1  t_001      2
2  t_002      2
3  t_003      3
4  t_004      3
